<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=93375ac6c2df0a4e72a52831fb7468590e0fa1fd0408a03e45d51d2aa3312609
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-30 09:16:26
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: 20.29 K (0.15%)
Current PnL: -22.90 L (-15.41%)
CY Booked + Current PnL: -11.46 L (-7.71%)
-------------------
Total profit:  97.94 K
Total loss:  -23.88 L
-------------------
Total Booked + Current PnL: 15.52 L (12.75%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.37%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.77 L (64.94%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.0%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,93.81,37.0,M-SC,2.45,83519.0,-17258.0,17347.0,0.41,-17.12,20.77,0.09,245.0,-0.99,0.61,9.73,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.94,48.0,H-MC,3.43,189830.0,25160.0,18167.0,0.14,15.28,9.57,26.31,89.0,1.38,1.39,24.48,X40N,NTT,AC
50,NATIONALUM,244.55,-45.32,54.0,H-MC,12.07,109175.0,8671.0,20438.0,1.02,8.63,18.72,28.96,79.0,0.42,0.80,45.52,MH,ATH,METALS
42,ITC,452.00,-38.77,46.0,H-LC,1.91,198143.0,-1995.0,22886.0,-0.50,-1.00,11.55,10.44,4.0,-0.09,1.45,4.51,X40,NTT,FMCG
84,VOLTAS,1530.00,-1.50,49.0,H-MC,2.89,206595.0,14853.0,22911.0,2.77,7.75,11.09,19.69,99.0,0.65,1.51,15.07,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1201.44,66.0,M-SC,14.08,177089.0,-4054.0,49514.0,3.40,-2.24,27.96,25.10,235.0,-0.08,1.30,41.92,XY24,NTT,HEALTHCARE
52,PGHH,17973.08,-27.24,65.0,H-MC,3.21,210750.0,9930.0,58841.0,2.95,4.94,27.92,34.25,80.0,0.17,1.54,10.40,X40,ATH,FMCG
84,VOLTAS,1530.00,-1.50,49.0,H-MC,2.89,206595.0,14853.0,22911.0,2.77,7.75,11.09,19.69,99.0,0.65,1.51,15.07,XY25,NTT,AC
61,SAMMAANCAP,326.00,-182.95,76.0,M-SC,7.12,94974.0,-7236.0,100625.0,2.61,-7.08,105.95,91.37,208.0,-0.07,0.70,54.17,XY25,NTT,FINANCE
53,QUESS,986.00,-43.25,39.0,M-SC,38.73,56382.0,-8624.0,159555.0,1.84,-13.27,282.99,232.18,198.0,-0.05,0.41,1.84,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-46.98,25.0,M-SC,1.27,89625.0,-1874.0,80627.0,-3.05,-2.05,89.96,86.07,223.0,-0.02,0.66,30.15,XR,NTT,DURABLES
28,HATHWAY,31.40,860.14,26.0,H-SC,2.01,82800.0,-25920.0,105603.0,-2.75,-23.84,127.54,73.29,161.0,-0.25,0.61,10.75,XR,NTT,ENTERTAINMENT
26,GREENPANEL,537.00,193.93,39.0,M-SC,0.99,131345.0,-51733.0,130715.0,-2.48,-28.26,99.52,43.14,230.0,-0.40,0.96,23.55,XY24,NTT,MISC
38,INDIGOPNTS,1408.00,101.45,29.0,M-SC,1.53,131936.0,-42623.0,42655.0,-2.44,-24.42,32.33,0.02,221.0,-1.00,0.97,14.76,OX40N,NTT,PAINTS
37,INDIAMART,4850.92,-56.13,22.0,H-SC,9.95,122695.0,-641.0,134400.0,-2.19,-0.52,109.54,108.45,119.0,-0.00,0.90,21.72,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4850.92,-56.13,22.0,H-SC,9.95,122695.0,-641.0,134400.0,-2.19,-0.52,109.54,108.45,119.0,-0.00,0.9,21.72,AR,ATH,MISC
32,HINDZINC,730.22,25.32,61.0,M-LC,3.30,204450.0,-626.0,113204.0,0.83,-0.31,55.37,54.89,52.0,-0.01,1.5,23.72,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,97.20,56.0,H-SC,10.80,124254.0,-18225.0,37748.0,-0.13,-12.79,30.38,13.70,163.0,-0.48,0.91,46.65,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.81,37.0,M-SC,2.45,83519.0,-17258.0,17347.0,0.41,-17.12,20.77,0.09,245.0,-0.99,0.61,9.73,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-68.16,49.0,H-SC,1.52,220131.0,-49536.0,85873.0,-0.81,-18.37,39.01,13.47,138.0,-0.58,1.61,11.34,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,37.0,H-SC,1.60,141140.0,-34763.0,76780.0,-0.36,-19.76,54.40,23.89,149.0,-0.45,1.03,21.89,OX40N,NTT,CERAMICS
66,SIS,528.0,2079.79,40.0,H-SC,4.02,87975.0,-23057.0,46662.0,-0.06,-20.77,53.04,21.26,156.0,-0.49,0.64,18.86,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,55.0,M-MC,11.15,228509.0,3547.0,164115.0,0.75,1.58,71.82,74.53,192.0,0.02,1.67,34.34,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.94,48.0,H-MC,3.43,189830.0,25160.0,18167.0,0.14,15.28,9.57,26.31,89.0,1.38,1.39,24.48,X40N,NTT,AC
84,VOLTAS,1530.00,-1.50,49.0,H-MC,2.89,206595.0,14853.0,22911.0,2.77,7.75,11.09,19.69,99.0,0.65,1.51,15.07,XY25,NTT,AC
17,CAMS,5211.76,-6.71,42.0,H-SC,1.81,106042.0,4038.0,39893.0,-0.34,3.96,37.62,43.06,122.0,0.10,0.78,21.75,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,53.0,M-LC,7.02,154689.0,13849.0,30644.0,1.47,9.83,19.81,31.59,66.0,0.45,1.13,38.93,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-45.32,54.0,H-MC,12.07,109175.0,8671.0,20438.0,1.02,8.63,18.72,28.96,79.0,0.42,0.80,45.52,MH,ATH,METALS
11,BANKINDIA,190.00,-29.19,57.0,H-MC,8.75,183653.0,3845.0,108576.0,0.54,2.14,59.12,62.52,88.0,0.04,1.34,36.95,XR,NTT,BANKS
37,INDIAMART,4850.92,-56.13,22.0,H-SC,9.95,122695.0,-641.0,134400.0,-2.19,-0.52,109.54,108.45,119.0,-0.00,0.90,21.72,AR,ATH,MISC
32,HINDZINC,730.22,25.32,61.0,M-LC,3.30,204450.0,-626.0,113204.0,0.83,-0.31,55.37,54.89,52.0,-0.01,1.50,23.72,X5K,ATH,METALS
86,WIPRO,420.00,-15.15,41.0,M-LC,5.58,149389.0,-1556.0,111011.0,0.55,-1.03,74.31,72.51,53.0,-0.01,1.09,4.93,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-6.42,29.0,H-LC,13.03,254345.0,-13020.0,78465.0,0.53,-4.87,30.85,24.48,48.0,-0.17,1.86,47.43,XY24,BTT,FINANCE
74,TCS,4476.75,-31.26,31.0,H-LC,12.27,273493.0,-72463.0,147331.0,0.46,-20.95,53.87,21.64,1.0,-0.49,2.00,0.46,X40,ATH,IT
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.03,217822.0,-24102.0,87368.0,-0.57,-9.96,40.11,26.15,8.0,-0.28,1.59,2.93,X40,ATH,IT
5,ASIANPAINT,3465.66,-17.51,33.0,H-LC,6.19,208208.0,-43560.0,96775.0,1.00,-17.30,46.48,21.13,27.0,-0.45,1.52,11.99,X40,ATH,PAINTS
83,VBL,671.64,-20.49,34.0,H-LC,8.78,286558.0,-29284.0,141961.0,-0.56,-9.27,49.54,35.67,5.0,-0.21,2.10,3.30,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-6.94,43.0,H-LC,1.63,230962.0,-15189.0,94394.0,-0.49,-6.17,40.87,32.18,16.0,-0.16,1.69,27.65,X200,ATH,IT
42,ITC,452.0,-38.77,46.0,H-LC,1.91,198143.0,-1995.0,22886.0,-0.50,-1.00,11.55,10.44,4.0,-0.09,1.45,4.51,X40,NTT,FMCG
65,SIEMENS,4671.5,-2.83,43.0,H-LC,2.39,156025.0,-30070.0,77544.0,0.78,-16.16,49.70,25.51,15.0,-0.39,1.14,14.98,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.41,42.0,H-LC,3.77,214032.0,3666.0,25106.0,-0.06,1.74,11.73,13.68,37.0,0.15,1.57,18.55,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-22.87,39.0,H-LC,3.99,205637.0,1137.0,40284.0,0.87,0.56,19.59,20.25,10.0,0.03,1.50,9.34,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,31.0,H-LC,12.27,273493.0,-72463.0,147331.0,0.46,-20.95,53.87,21.64,1.0,-0.49,2.00,0.46,X40,ATH,IT
80,UNITDSPR,1644.00,-14.09,46.0,H-LC,7.19,225268.0,-8876.0,57488.0,0.09,-3.79,25.52,20.77,86.0,-0.15,1.65,2.87,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.03,217822.0,-24102.0,87368.0,-0.57,-9.96,40.11,26.15,8.0,-0.28,1.59,2.93,X40,ATH,IT
83,VBL,671.64,-20.49,34.0,H-LC,8.78,286558.0,-29284.0,141961.0,-0.56,-9.27,49.54,35.67,5.0,-0.21,2.10,3.30,X40N,ATH,FMCG
42,ITC,452.00,-38.77,46.0,H-LC,1.91,198143.0,-1995.0,22886.0,-0.50,-1.00,11.55,10.44,4.0,-0.09,1.45,4.51,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.83,43.0,H-LC,2.39,156025.0,-30070.0,77544.0,0.78,-16.16,49.70,25.51,15.0,-0.39,1.14,14.98,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,46.0,H-LC,1.91,198143.0,-1995.0,22886.0,-0.50,-1.00,11.55,10.44,4.0,-0.09,1.45,4.51,X40,NTT,FMCG
19,CIPLA,1795.00,-22.87,39.0,H-LC,3.99,205637.0,1137.0,40284.0,0.87,0.56,19.59,20.25,10.0,0.03,1.50,9.34,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-17.51,33.0,H-LC,6.19,208208.0,-43560.0,96775.0,1.00,-17.30,46.48,21.13,27.0,-0.45,1.52,11.99,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,42.0,H-LC,3.77,214032.0,3666.0,25106.0,-0.06,1.74,11.73,13.68,37.0,0.15,1.57,18.55,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,31.0,H-LC,12.27,273493.0,-72463.0,147331.0,0.46,-20.95,53.87,21.64,1.0,-0.49,2.00,0.46,X40,ATH,IT
40,INFY,2275.00,-21.03,39.0,H-LC,8.20,307892.0,-4996.0,176668.0,0.26,-1.60,57.38,54.87,3.0,-0.03,2.25,4.95,X40,BTT,IT
42,ITC,452.00,-38.77,46.0,H-LC,1.91,198143.0,-1995.0,22886.0,-0.50,-1.00,11.55,10.44,4.0,-0.09,1.45,4.51,X40,NTT,FMCG
83,VBL,671.64,-20.49,34.0,H-LC,8.78,286558.0,-29284.0,141961.0,-0.56,-9.27,49.54,35.67,5.0,-0.21,2.10,3.30,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,46.0,H-LC,7.68,249768.0,-11851.0,131053.0,0.58,-4.53,52.47,45.57,7.0,-0.09,1.83,6.28,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,41.0,L-SC,40.17,77568.0,-35981.0,76032.0,-0.07,-31.69,98.02,35.27,268.0,-0.47,0.57,89.85,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,36.0,H-SC,12.31,90330.0,-7650.0,29249.0,-0.51,-7.81,32.38,22.05,152.0,-0.26,0.66,31.30,XR,ATH,FINANCE
61,SAMMAANCAP,326.00,-182.95,76.0,M-SC,7.12,94974.0,-7236.0,100625.0,2.61,-7.08,105.95,91.37,208.0,-0.07,0.70,54.17,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-45.32,54.0,H-MC,12.07,109175.0,8671.0,20438.0,1.02,8.63,18.72,28.96,79.0,0.42,0.80,45.52,MH,ATH,METALS
78,UJJIVANSFB,60.00,97.20,56.0,H-SC,10.80,124254.0,-18225.0,37748.0,-0.13,-12.79,30.38,13.70,163.0,-0.48,0.91,46.65,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1201.44,66.0,M-SC,14.08,177089.0,-4054.0,49514.0,3.40,-2.24,27.96,25.10,235.0,-0.08,1.30,41.92,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.0,-182.95,76.0,M-SC,7.12,94974.0,-7236.0,100625.0,2.61,-7.08,105.95,91.37,208.0,-0.07,0.70,54.17,XY25,NTT,FINANCE
60,SAIL,228.0,45.15,55.0,M-MC,11.15,228509.0,3547.0,164115.0,0.75,1.58,71.82,74.53,192.0,0.02,1.67,34.34,XY24,BTT,STEEL
7,ATULAUTO,844.0,3527.03,47.0,M-SC,7.42,113068.0,-34300.0,86113.0,0.25,-23.27,76.16,35.16,236.0,-0.40,0.83,15.49,XY24,NTT,AUTO
69,SURYODAY,240.0,46.93,57.0,H-SC,0.99,135660.0,-43411.0,109138.0,0.08,-24.24,80.45,36.71,135.0,-0.40,0.99,34.60,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.23
1,25,43.73
2,50,75.05


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.72
LC    32.53
MC    23.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.37
X40      14.79
X40N     11.45
XR       11.05
XY25     10.29
AR        8.20
OX40N     7.87
X200      1.69
MH        1.62
X5K       1.50
SR        1.19
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.20
H-LC    25.97
H-MC    20.82
M-SC    13.07
M-LC     5.50
M-MC     2.65
L-SC     1.45
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.13,-6.34,41.44
IT,12.69,-21.08,87.09
FINANCE,10.17,-18.10,69.62
MISC,6.95,-20.97,88.23
BANKS,6.26,-16.46,77.73
PAINTS,5.63,-20.03,37.93
ELECTRICAL,5.48,-9.88,48.43
INSURANCE,3.77,-7.50,47.68
AC,3.61,1.84,14.60


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3232546.0,22
XR,1399926.0,14
AR,1212271.0,9
X40,826230.0,10
X40N,641638.0,9
OX40N,570437.0,10
XY25,434739.0,7
SR,258395.0,2
X5K,113204.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3805985.0,29
M-SC,1461827.0,17
H-LC,1269687.0,15
H-MC,1261581.0,15
M-LC,398362.0,4
M-MC,315912.0,2
L-SC,262922.0,3
L-MC,61058.0,1
L-LC,39729.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1246492.0      6
M-SC       XY24       836622.0      7
H-SC       AR         831527.0      5
           XR         815841.0      7
H-MC       XY24       580908.0      4
H-LC       X40        531028.0      5
H-SC       X40N       317836.0      4
M-MC       XY24       315912.0      2
H-SC       OX40N      263049.0      4
           SR         258395.0      2
H-LC       X40N       239733.0      3
H-MC       X40        217573.0      4
H-LC       AR         208597.0      2
H-MC       XY25       176915.0      2
M-LC       XY24       174147.0      2
L-SC       XR         174007.0      2
M-SC       AR         172147.0      2
           OX40N      145371.0      4
           XR         129433.0      2
H-LC       XY25       117470.0      3
M-LC       X5K        113204.0      1
           XR         111011.0      1
H-MC       XR         108576.0      1
M-SC       XY25       100625.0      1
H-LC       X200        94394.0      1
L-SC       OX40N       88915.0      1
H-MC       X40N        84069.0      2
H-LC       XY24        78465.0      1
M-SC       X40         77629.0      1
H-MC       OX40N       73102.0      1
H-SC       MH          72845.0      1
L-MC       XR          61058.0      1
L-LC       XY25        39729.0      1
H-MC       MH          20438.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 51.0 seconds
